# 09-2 순환 신경망으로 IMDB 리뷰 분류하기
### 텐서플로를 이용한 순환 신경망을 영화 리뷰 데이터셋에 적용하여, 리뷰를 긍정과 부정으로 분류
---
<br/>
💡용어 정리💡  

❓ 토큰 ➡️ 숫자에 매핑되게끔 분리된 단어  
 (하나의 샘플은 여러 개의 토큰으로 이뤄져 있고, 1개의 토큰이 하나의 타임스텝에 해당됨!)  

imdb 데이터 적재

In [ ]:
from tensorflow.keras.datasets import imdb
# num_words=500 --> 어휘 사전에 500개의 단어만 탑재(어휘 사전에 없는 단어들은 전부 2로 표시!)
(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=500)

17473536/17464789 [==============================] - 0s 0us/step


훈련 세트 및 테스트 세트 크기 확인  
- IMDB 리뷰 텍스트의 길이는 제각각 --> 고정 크기의 2차원 배열 보다는 리뷰마다 별도의 파이썬 리스트로 저장!  
- 하나의 리뷰 = 하나의 샘플  

In [ ]:
print(train_input.shape, test_input.shape)

(25000,) (25000,)


첫 번째 리뷰의 길이 출력

In [ ]:
print(len(train_input[0]))

218


두 번째 리뷰의 길이 출력

In [ ]:
print(len(train_input[1]))

189


첫 번째 리뷰에 담긴 내용 출력

In [ ]:
print(train_input[0])

[1, 14, 22, 16, 43, 2, 2, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 2, 112, 50, 2, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 2, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 2, 38, 76, 15, 13, 2, 4, 22, 17, 2, 17, 12, 16, 2, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 2, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 2, 36, 71, 43, 2, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]


타깃 데이터 출력  
- 리뷰가 긍정인지? 부정인지? --> 0 혹은 1 (이진분류!)  


In [ ]:
print(train_target[:20])

[1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1]


훈련 세트에서 검증 세트 분리

In [ ]:
from sklearn.model_selection import train_test_split
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

각 리뷰의 길이를 계산해 넘파이 배열에 저장  
--> for 평균적인 리뷰의 길이와 가장 짧은 리뷰의 길이, 가장 긴 리뷰의 길이 확인

In [ ]:
import numpy as np
lengths = np.array([len(x) for x in train_input])